In [ ]:
%pip install -q langchain langchain-groq pydantic faiss-cpu typing langchain_community langchain-huggingface

In [1]:
import os

In [4]:
import os
import json
import csv
import pandas as pd
from typing import Dict, List, Optional
from langchain_groq.chat_models import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.output_parsers import PydanticOutputParser
from langchain_community.document_loaders import csv_loader
from pydantic import BaseModel, Field
import logging

In [3]:
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [7]:
path = r"/content/drive/MyDrive/__SHL__/shl_catalog_edited.csv"
data = pd.read_csv(path)

In [ ]:
data.head()

In [10]:
data=data.drop("Unnamed: 0",axis = 1)

In [11]:
data.columns

Index(['Assessment Name', 'Assessment URL', 'Remote Testing Support',
       'Adaptive/IRT Support', 'Duration', 'Ability&Aptitude',
       'Biodata&SituationalJudgement', 'Competencies', 'Developemnt&360',
       'AssessmentExercies', 'Knowledge&Skills', 'Personality&Behavior',
       'Simulation', 'Description'],
      dtype='object')

## **LangChain Model with Message History**







In [12]:

def get_test_type(row):
  test_types = []
  cols = ['Ability&Aptitude',
       'Biodata&SituationalJudgement', 'Competencies', 'Developemnt&360',
       'AssessmentExercies', 'Knowledge&Skills', 'Personality&Behavior',
       'Simulation']
  for col in cols:
    if row[col ] == 1:
      test_types.append(col)
  return ",".join(test_types)

def create_document(row):
  metadata = {
        "name": row["Assessment Name"],
        "url": row["Assessment URL"],
        "remote_support": row["Remote Testing Support"],
        "adaptive_support": row["Adaptive/IRT Support"],
        "duration": row["Duration"],
        "test_type": get_test_type(row)
    }

  return Document(page_content = row['Description'], metadata=metadata)

docs = [create_document(row) for _, row in data.iterrows()]

In [ ]:
import pprint
pprint.pprint(docs)

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)

In [38]:
memory = ConversationBufferMemory(memory_key="chat_history", output_key="answer", return_messages=True)
llm = ChatGroq(model="llama-3.3-70b-versatile",
               api_key = GROQ_API_KEY)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ca8d4046e90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ca8cff38110>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [39]:
QA_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 10}),
    memory=memory,
    return_source_documents=True,
    verbose = True
)

In [31]:
def format_response(response_docs):
    """Format the documents into a structured output."""
    results = []
    for doc in response_docs:
        meta = doc.metadata
        results.append({
            "Assessment Name": meta.get("name"),
            "URL": meta.get("url"),
            "Remote Support": meta.get("remote_support"),
            "Adaptive Support": meta.get("adaptive_support"),
            "Duration": meta.get("duration"),
            "Test Type": meta.get("test_type")
        })
    return results

In [ ]:
def ask_question(query: str, history=[]):
    response = QA_chain.invoke({"question": query})
    docs = response.get("source_documents", [])
    formatted_results = format_response(docs)
    formatted_results = formatted_results[:10]

    return {
        "answer": response["answer"],
        "recommendations": formatted_results
    }

In [ ]:
def display_results(results):
    if not results:
        return "No matching tests found."
    df = pd.DataFrame(results)
    return df

In [ ]:
def get_recommendations(query):
    try:
        response = ask_question(query)
        print(f"Answer: {response['answer']}\n")
        print("Recommended Tests:")
        return display_results(response["recommendations"])
    except Exception as e:
        logging.error(f"Error getting recommendations: {str(e)}")
        return f"Error: {str(e)}"

In [ ]:
if __name__ == "__main__":
    query = input("Enter your job description or requirements: ")
    recommendations = get_recommendations(query)
    print(recommendations)

In [41]:
QUERY = "I need to hire a software engineer with strong leadership skills and experience in Agile development."
JOB_DESCRIPTION = """Software Engineer - Team Lead

Our company is seeking a highly motivated and experienced Software Engineer to join our team as a Team Lead. The ideal candidate will possess a strong background in software development, excellent leadership abilities, and a proven track record of working effectively in Agile environments.

Responsibilities:

* Lead a team of 5-7 software engineers in the design, development, and implementation of high-quality software solutions.
* Provide technical guidance and mentorship to team members.
* Participate in Agile sprint planning, daily stand-ups, sprint reviews, and retrospectives.
* Write clean, efficient, and well-documented code in Java and Python.
* Collaborate with product managers, designers, and other stakeholders to define project requirements.
* Ensure adherence to coding standards and best practices.

Qualifications:

* Bachelor's degree in Computer Science or a related field.
* 5+ years of professional software development experience.
* 2+ years of experience in a team lead or similar role.
* Strong proficiency in Java and Python.
* Experience with Agile development methodologies.
* Excellent communication and interpersonal skills.
* Ability to work independently and as part of a team.

Preferred Qualifications:

* Master's degree in Computer Science.
* Experience with cloud computing platforms (e.g., AWS, Azure).
* Contributions to open-source projects."""

In [ ]:
query = JOB_DESCRIPTION
recommendations = get_recommendations(query)
print(recommendations)

In [ ]:
vectorstore.save_local("/content/drive/MyDrive/__SHL__/vectorstore/shl_faiss")